## Subsetting Code:

In [3]:
%pylab notebook
import netCDF4
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
#from cetbtools.ease2conv import Ease2Transform
import subprocess
from subprocess import call
import shlex
import os
import sys
import glob
from nco import Nco
import ease2conv as e2
import rasterio

Populating the interactive namespace from numpy and matplotlib


In [2]:
from nco import Nco
nco = Nco()

In [3]:
path = os.getcwd()
path

'/Users/williamnorris/Team_GISProg'

In [4]:
list1 = glob.glob('*19H*')

In [6]:
nco.ncrcat(input = list1, output = 'out1.nc')

KeyboardInterrupt: 

## File Setup:

In [4]:
wget = path + "/data/wget/"
path19 = path + "/data/Subsetted_19H/"
path37 = path + "/data/Subsetted_37H/"
env_path = '/anaconda3/envs/learning/bin/'

if not os.path.exists(wget):\
    os.makedirs(wget)
if not os.path.exists(path19):
    os.makedirs(path19)
if not os.path.exists(path37):
    os.makedirs(path37)

## Input Study Area

In [12]:
def get_xy(ll_ul, ll_lr):
    N6 = e2.Ease2Transform("EASE2_N6.25km")
    N3 = e2.Ease2Transform("EASE2_N3.125km")
    # get x,y for 6.25
    row, col = N6.geographic_to_grid(ll_ul[0], ll_ul[1])
    x6ul, y6ul = N6.grid_to_map(row,col)
    row, col = N6.geographic_to_grid(ll_lr[0], ll_lr[1])
    x6lr, y6lr = N6.grid_to_map(row, col)
    # get x,y for 3.125
    row, col = N6.geographic_to_grid(ll_ul[0], ll_ul[1])
    x3ul, y3ul = N6.grid_to_map(row, col)
    row, col = N6.geographic_to_grid(ll_lr[0], ll_lr[1])
    x3lr, y3lr = N6.grid_to_map(row, col)
    list_6 = [x6ul, y6ul, x6lr, y6lr]
    list_3 = [x3ul, y3ul, x3lr, y3lr]
    return list_3, list_6

In [13]:
def subset(list19, list37, list3, list6, path19, path37, wget):
    # for loop to take each file in the list and subset it's location 
    for file in list19:
        outfile = path19 + file
        infile = wget + file
        cmd = env_path + 'ncks -d x,'+list6[0] +','+ list6[2]+' -d y,'+list6[4]+ ',' + list6[1]+' -v TB ' + infile + " " + outfile
        print("Next in : %s" % file)
        print("Next out: %s" % outfile)
        # print("next unl out: %s" % unlfile)
        subprocess.call(shlex.split(cmd), shell = False)
        os.remove(infile)
        print("Done")


    for file in list37: 
        outfile = path37 + file 
        infile = wget + file
        cmd = env_path + 'ncks -d x,'+list3[0] +','+ list3[2]+' -d y,'+list3[4]+ ',' + list3[1]+' -v TB ' + infile + " " + outfile
        print("Next in: %s" % infile)
        print("Next out: %s" % outfile)
        subprocess.call(shlex.split(cmd), shell = False)
        os.remove(infile)
        print("Done")   

In [14]:
# upper left 
lat_lon_ul = [62.27, -140.17]

# lower right 
lat_lon_lr =[73.64, -166.08]

list_3, list_6 = get_xy(lat_lon_ul, lat_lon_lr)

In [15]:
os.chdir(wget)
# Make a list of the files to concatenate together for 19H
list19 = sort(glob.glob("*19H-M-SIR*"))
# Make list for the 37GHz
list37 = sort(glob.glob("*37H-M-SIR*"))

In [16]:
subset(list19, list37, str(list_3), str(list_6), path19, path37, wget)